# Bitácora Creación CNN Géstos de Mano
El objetivo de este Notebook es registrar y compartir las experiencias adquiridas al desarrollar una CNN destinada al reconicimiento de gestos de mano.

* Curso: INFO257 Inteligencia Artificial
* Docente: Pablo Huijse

#### Integrantes
* Eduardo Hopperdietzel
* Sebastián Lara

# Creación de Nuestro Modelo

Tal como nos recomendó el profesor, comenzamos imitando la estructura del modelo Lenet5, luego fuimos quitando y añadiendo neuronas/capas, y jugando con los tamaños de kernel, stride y cantidad de padding.

Pronto nos dimos cuenta que aumentar el número de neuronas mejoraba considerablemente la precisión del modelo, no así la cantidad de capas.

### Neuronas

Comenzamos a decidir el número de neuronas teniendo en cuenta el tamaño de las imágenes, la cantidad canales por pixel, cantidad de clases, cantidad de patrones o formas de los dedos, etc, y decidimos por ejemplo que la salida de una capa sea el triple de la entrada anterior (para que pueda aprender patrones por cada color RGB). Luego simplemente fuimos probando con la cantidad de neuronas por capa, partiendo inicialmente con 4, luego 8, 16, finalizando con 32 neuronas, con la que obtuvimos mejores resultados.

### Capas
Partimos utilizando entre 1 y 3 capas con 16 neuronas cada una aprox, obteniendo resultados regualares, luego probamos con muchas capas 8 a 16 sin muchas neuronas y el resultado fue peor, por lo tanto volvimos a utilizar pocas con muchas neuronas (16 a 256), y fuimos añadiendo capas una a una hasta que obtuvimos los mejores resultados con 5.
Luego fuimos reduciendo poco a poco la cantidad de neuronas hasta el punto en que notamos comenzó a empeorar nuevamente.

### Kernel
Probamos diversos tamaños de kernel por capa, y nos dimos cuenta que obteniamos mejores resultados comenzando con tamaños grandes en las primeras y luego pequeños para las últimas.
Tambien notamos que al usar un kernel relativamente grande (11) en la primera capa, tras entrenar los modelos podíamos ver que se generaban patrones, por lo tanto lo mantuvimos.

*Filtros de la Primera Capa*
![](https://i.imgur.com/Yp5Gekp.png)

( Creemos que podríamos eliminar algunas neuronas dado que algunos filtros solo parecieran almacenar ruido )
### Pooling y Stride
Aplicamos MaxPooling y Stride mayores a las capas con el fin de simplificar el modelo, fuimos aumentando hasta que creemos no se vió afectado la precisión del modelo.
También aplicamos AveragePooling en la última capa de convulsión para suavisar la salida a la capa fully connected.

### Capa Fully Connected
Decidimos mantener la cantidad de capas fully connected de Lenet5, ya que obtuvimos horrendos resultados con otras cantidades, sin embargo aumentar el número de neuronas resultó ser muy efectivo.

### Función de Activación

Probamos la función ReLU y Tanh, y no notamos mucha diferencia, pero finalmente optamos por ReLU ya que en varios sitios web la recomendaban para CNNs.

### Tamaño del Batch
Probamos distintos tamaños, desde 1 hasta 200, y observamos que con tamaños pequeños la loss de validación nunca bajaba ni el acurracy subia.
Con tamaños un poco más grandes 16-32 la loss y acurracy de entrenamiento y validación, presentaba una mejora coherente en cada época.

![](https://i.imgur.com/QVxVkG5.png)

### Checkpoints
Inicialmente utilizamos la función de checkpoint de ignite, almacenando los modelos con menor loss de entrenamiento, pero tras visualizar las gráficas, nos surgieron dudas sobre si era la mejor métrica para tener en cuenta. Por lo tanto, decidimos almacenar los modelos de todas las épocas ( ya que con nuestras últimas estructuras pesaban aprox 5mb ), y luego probabamos cáda uno en el dataset de prueba y validación, midiendo su acurracy promedio.

*( Casi siempre el mejor modelo de todas las épocas lo encontrabamos entre la 70 y 120 )*

### Pre-Procesamiento de Imágenes
Comenzamos a entrenar los modelos sin añadir transformaciones al dataset de entrenamiento, luego probamos reduciendo las resoluciones a 100px x 100px, 64px x 64px, etc y pudimos notar una leve mejora, sin embargo luego de observar las fotos del dataset de prueba, nos dimos cuenta que la mayoría de los brazos se encontraban inclinadas con un ángulo de aprox 20º, por lo tanto les aplicamos una rotación random de -20º a 20º, y un flip horizontal también random al dataset de entrenamiento y nos sorprendió la mejora.
También intentamos reducir el bit depth de los colores (para acelerar el entrenamiento) pero no encontramos la manera.

### Estructura Final
Finalmente tras probar muchas combinaciones la mejor estructura que obtuvimos fue la siguiente:

```python
class HandModel(torch.nn.Module):
    def __init__(self):
        super(type(self), self).__init__()
        self.c1 = nn.Conv2d(3, 32, kernel_size=11, stride=4, padding=2)
        self.c2 = nn.Conv2d(32,96, kernel_size=5, padding=2)
        self.c3 = nn.Conv2d(96,192, kernel_size=3, padding=1)
        self.c4 = nn.Conv2d(192, 128, kernel_size=3, padding=1)
        self.c5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.ac = nn.ReLU(inplace=True)
        self.mp = nn.MaxPool2d(kernel_size=3, stride=2)
        self.ap = nn.AdaptiveAvgPool2d((3, 3))
        self.f1 = nn.Linear(1152, 512)
        self.f2 = nn.Linear(512, 512)
        self.f3 = nn.Linear(512, 4)
    def forward(self, x):
        x = self.mp(self.ac(self.c1(x)))
        x = self.mp(self.ac(self.c2(x)))
        x = self.ac(self.c3(x))
        x = self.ac(self.c4(x))
        x = self.mp(self.ac(self.c5(x)))
        x = self.ap(x)
        x = torch.flatten(x, 1)
        x = self.ac(self.f1(x))
        x = self.ac(self.f2(x))
        x = self.f3(x)
        return x
```



# Rendimiento

## Métricas

<img src="https://i.imgur.com/YlmM012.png" width="400"/>
<center><i>(Resultados sobre el dataset de prueba)</i></center>

Podemos observar que nuestro modelo es excelente detectando fondos vacíos y manos con un dedo, sin embargo su precisión es mucho peor para detectar manos con 3 dedos.

Dado que seleccionamos el modelo con mayor acurracy promedio sobre el dataset de prueba, medimos también su rendimiento sobre el dataset de validación, para verificar que no hayamos seleccionado un modelo que solo funcione con el de prueba.


<img src="https://i.imgur.com/jzxzOoS.png" width="400"/>
<center><i>(Resultados sobre el dataset de validación)</i></center>

Nos alegró comprobar que su rendimiento también es bueno, e incluso mejor que en el de prueba.

## Matriz de Confusión

#### Dataset de Prueba
![](https://i.imgur.com/fiSHloS.png)

Con la matriz de confusión podemos apreciar que en general tiende a confundir las imágenes de manos con los fondos vacíos, siendo las imagenes de 3 dedos las más problemáticas.

## Imágenes Mal Clasificadas

![](https://i.imgur.com/KYZEacD.png)

Tomando algunos ejemplos mal clasificados, podemos ver que no es evidente la razón de por que confunde las imágenes de 3 dedos con los fondos vacíos.

Creemos que probablemente se deba a que en el dataset de entrenamiento, existan imágenes de 3 dedos con estos mismos fondos.



# Modelo ResNet18

Seleccionamos el modelo ResNet18 como nos recomendó el profesor y le aplicamos transferencia de aprendizaje, entrenando solamente la última capa de su red fully conected.

### Tamaño Batch
Dado que no podiamos modificar la estructura del modelo, simplemente lo fuimos entrenando con distintos tamaños de batch.
El modelo con el que obtuvimos mejores resultados, lo entrenamos con un batch de 32, pero tambien probamos con otros como 4, 16, 64, 128 y 200.

![](https://i.imgur.com/58JYske.png)

Con tamaños menores a 32, el modelo parecía perder el control sobre las 50 épocas, donde la loss de entrenamiento y validación comenzaba a subir, y el acurracy de ambas a bajar. Y con tamaños mayores a 64 se mantenía estable durante muchas, sin generar mejoras, con lo que pensamos llegaba a un mínimo local.

### Checkpoints
Debido a que el tamaño del modelo pesa aproximadamente 45mb, no utilizamos la misma estrategia que con el nuestro.
Usamos la función de ignite, almacenando el de menor loss de entrenamiento.

### Pre-Procesamiento de Imágenes
Al igual que en nuestro modelo aplicamos una rotación random de -20º a 20º, y un flip horizontal también random.
Cambiamos la resolución a 224px x 224px y aplicamos la siguiente normalización:

```python
[0.485, 0.456, 0.406],
[0.229, 0.224, 0.225]
```

Ya que así lo indicaba su documentación. Obviamente también aplicamos éstas últimas dos transformaciones al dataset de prueba, al momento de evaluarlo.

## Métricas

<img src="https://i.imgur.com/2xf3t9R.png" width="400"/>
<center><i>(Resultados sobre el dataset de prueba)</i></center>

Nos sorprendió ver que el rendimiento que pudimos obtener de él fue mucho peor que con el nuestro.
Sin embargo parece tener las mismas vritudes/debiliades, ( bueno con los fondos vacíos y malo con las fotos de 3 y 2 dedos ).

## Matriz de Confusión

#### Dataset de Prueba
![](https://i.imgur.com/dmgWLgn.png)

Con la matriz de confusión tambíen podemos apreciar que en general tiende a confundir las imágenes de manos con los fondos vacíos, pero también entre ellas mismas.

## Imágenes Mal Clasificadas

![](https://i.imgur.com/4coXJYg.png)

De la misma forma que con nuestro modelo, cuesta visualzar la razón de por que esta confundiendo las imágenes.



# Conclusión

Podemos concluir que aplicar transferencia de aprendizaje sobre la última capa de un modelo pre entrenado no genera resultados tan buenos como entrenar un modelo propio o completo.
También creemos que para problemas con pocas clases no conviene utilizar muchas capas pero si muchas neuronas.
Y por último que comenzar con tamaños de kernel grandes y luego pequeños en las capas de convulsión genera buenos resultados.

